In [137]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score,precision_score, recall_score, roc_auc_score
from sklearn.decomposition import PCA
from sklearn.utils import shuffle
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.mixture import GaussianMixture
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer
import tensorflow.keras
import keras.metrics
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Input
from keras.models import Sequential
from keras.callbacks import History 
from keras.utils import plot_model
from keras.optimizers import SGD
from keras.models import Model
import featuretools as ft
from sklearn.semi_supervised import LabelSpreading, LabelPropagation
from keras.models import Model 
from keras.layers import Input, Dense 
from keras.utils import np_utils 


In [138]:
path = "PTSD.xlsx"
df = pd.read_excel(path)
#df = df[~df["PCL_Strict3"].isna()]


In [139]:
features = ["ID" ,"age", "highschool_diploma", "Hebrew", "dyslexia", "ADHD", "T1ETBE", "T1Acc1t",
                         "T1Acc1n", "T1bias", "T2Acc1t", "T2Acc1n", "T2bias", "state1", "state2", "trait1",
                         "trait2", "lot1", "lot2", "phq1", "phq2", "PCL1", "PCL2", "cd_risc1", "ptgi2",
                         "active_coping1", "planning1", "positive_reframing1", "acceptance1", "humor1",
                         "religion1", "emotional_support1", "instrumental_support1", "self_distraction1",
                         "denial1", "venting1", "substance_use1", "behavioral_disengagement1", "self_blame1",
                         "active_coping2", "planning2", "positive_reframing2", "acceptance2", "humor2",
                         "religion2", "emotional_support2", "instrumental_support2","self_distraction2",
                         "denial2", "venting2", "substance_use2", "behavioral_disengagement2", "self_blame2",
                         "trauma_history8_1", "military_exposure_unit", "HML_5HTT", "HL_MAOA", "HML_NPY",
                         "COMT_Hap1_recode", "COMT_Hap2_recode", "COMT_Hap1_LvsMH", "HML_FKBP5", "Ashken_scale",
                         "Sephar_scale", "Unknown", 'terror_p1', 'terror_i1', 'mva_p1', 'mva_i1',                   
                         'violent1', 'sexual1', 'rockets_p1', 'rockets_i1', 'trauma_history6_1',
                        'terror_p2','terror_i2','mva_p2', 'mva_i2', 'violent2', 'sexual2', 'rockets_p2',
                        'rockets_i2', 'trauma6t2', 'trauma8t2', 'military_exp18_1','military_exp18_t2',
                        'commanders18','commanders20', 'commanders22', 't1bias_1_zero', 'state1_zero',
                        'trait1_zero', 'PHQ1_zero', 'PCL1_zero', 'depression_clinical2', 'avoid_bias',
                        'ptsd1_clini', 'avoidance_cop', 'clinical_depression15', 'avoidance_compa',
                        'resilience_compa', 'combat_compa', 'emotional_cop1n', 'avoidance_cop2']

In [140]:
numerical_features = [ "T1ETBE", "T1Acc1t", "T1Acc1n", "T1bias", "T2Acc1t","T2Acc1n", "T2bias", "state1", "state2",
                        "trait1", "trait2", "lot1", "lot2", "phq1", "phq2", "cd_risc1", "PCL1", "PCL2"]
categorical_features = [ "age", "highschool_diploma", "Hebrew", "dyslexia", "ADHD",   "ptgi2",
                    "active_coping1", "planning1", "positive_reframing1", "acceptance1", "humor1",
                    "religion1", "emotional_support1", "instrumental_support1", "self_distraction1",
                    "denial1", "venting1", "substance_use1", "behavioral_disengagement1", "self_blame1",
                    "active_coping2", "planning2", "positive_reframing2", "acceptance2", "humor2",
                    "religion2", "emotional_support2", "instrumental_support2", "self_distraction2",
                    "denial2", "venting2", "substance_use2", "behavioral_disengagement2", "self_blame2",
                    "trauma_history8_1", "military_exposure_unit", "HML_5HTT", "HL_MAOA", "HML_NPY",
                    "COMT_Hap1_recode", "COMT_Hap2_recode", "COMT_Hap1_LvsMH", "HML_FKBP5", "Ashken_scale",
                    "Sephar_scale", "Unknown", 'terror_p1', 'terror_i1', 'mva_p1', 'mva_i1',                   
                    'violent1', 'sexual1', 'rockets_p1', 'rockets_i1', 'trauma_history6_1', 'terror_p2','terror_i2',
                    'mva_p2', 'mva_i2', 'violent2', 'sexual2', 'rockets_p2', 'rockets_i2', 'trauma6t2', 'trauma8t2',
                    'military_exp18_1','military_exp18_t2', 'commanders18','commanders20','commanders22', 
                    't1bias_1_zero', 'state1_zero', 'trait1_zero', 'PHQ1_zero', 'PCL1_zero', 'depression_clinical2',
                    'avoid_bias', 'ptsd1_clini', 'avoidance_cop', 'clinical_depression15', 'avoidance_compa',
                    'resilience_compa', 'combat_compa', 'emotional_cop1n', 'avoidance_cop2', 'avoidance_cop']


In [141]:
df.dropna(thresh=0.5, axis=1,inplace=True)
df.dropna(thresh=0.9, axis=0,inplace=True)

In [142]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
df[numerical_features] = imp.fit_transform(df[numerical_features])

imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
df[categorical_features] = imp.fit_transform(df[categorical_features])
Y = df["PCL_Strict3"]
df = df[features]
df = df - df.mean()




In [143]:
es = ft.EntitySet(id = 'ID')
es = es.entity_from_dataframe(entity_id='ID', dataframe = df)
new_features, new_feature_names = ft.dfs(entityset = es, target_entity = 'ID',  max_depth = 3)

2019-01-26 15:51:50,174 featuretools.entityset - WARNING    ('Using first column as index. ', 'To change this, specify the index parameter')


In [144]:
df = new_features
new_feature_names = df.columns
df["PCL_Strict3"] = Y.values

In [145]:
X = df[~df["PCL_Strict3"].isna()][new_feature_names]
Y = df[~df["PCL_Strict3"].isna()]["PCL_Strict3"]

In [146]:
new_feature_names

Index(['age', 'highschool_diploma', 'Hebrew', 'dyslexia', 'ADHD', 'T1ETBE',
       'T1Acc1t', 'T1Acc1n', 'T1bias', 'T2Acc1t',
       ...
       'depression_clinical2', 'avoid_bias', 'ptsd1_clini', 'avoidance_cop',
       'clinical_depression15', 'avoidance_compa', 'resilience_compa',
       'combat_compa', 'emotional_cop1n', 'avoidance_cop2'],
      dtype='object', length=103)

In [147]:
np.sum(df[df["PCL_Strict3"].isna()][new_feature_names])

age                      -17.830462
highschool_diploma       -19.521306
Hebrew                    -3.404352
dyslexia                   2.241160
ADHD                      13.145059
T1ETBE                  -319.917940
T1Acc1t                    0.013184
T1Acc1n                   -0.028026
T1bias                  -361.705024
T2Acc1t                   -0.074680
T2Acc1n                   -0.251227
T2bias                   773.998251
state1                   308.648109
state2                   334.416368
trait1                   232.267073
trait2                   330.670829
lot1                      13.822629
lot2                     -21.333296
phq1                     260.511176
phq2                     293.167456
PCL1                     456.535711
PCL2                     529.194120
cd_risc1                   3.276339
ptgi2                    505.905839
active_coping1            -2.641886
planning1                  6.594742
positive_reframing1       -4.353581
acceptance1               -8

In [148]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.1, random_state=271828, stratify=Y)
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_train, y_train, test_size = 0.05, random_state=271828, stratify=y_train)


In [149]:
def create_data(X_train, y_train):
    X_train_3 = X_train[y_train==1]
    y_train_3 = y_train[y_train==1]
    X_train_4 = X_train[y_train==0][:20:]
    y_train_4 = y_train[y_train==0][:20:]
    X_train_5 = np.vstack((X_train_4, X_train_3))
    y_train_5 =  np.hstack((y_train_4, y_train_3))
    sm = SMOTE(random_state=27)
    X_train_6, y_train_6 = sm.fit_sample(X_train_5, y_train_5.ravel())
    X_train_6 = X_train_6[y_train_6==0]
    y_train_6 = y_train_6[y_train_6==0]
    return X_train_6, y_train_6

In [150]:
def semi(X_train, y_train, X_test, pca):
    
    xxx = df[df["PCL_Strict3"].isna()][new_feature_names]
    xxx = pca.transform(xxx)
    X_train_2 = np.vstack((X_train, xxx))
    
    unlabeled = df[df["PCL_Strict3"].isna()]['PCL_Strict3'].fillna(-1)
    
    y_train_2 = np.hstack((y_train, unlabeled))
    
    clf = LabelSpreading(gamma=20, n_neighbors=150,alpha=0.0005,kernel='knn')
    
    clf.fit(X_train_2, y_train_2)
    
    
    X_train = np.hstack((X_train, clf.predict(X_train).reshape(-1,1)))
    X_test = np.hstack((X_test, clf.predict(X_test).reshape(-1,1)))
    return X_train, X_test

In [151]:
def auto(X_train, X_test, pca):
    
    xxx = df[df["PCL_Strict3"].isna()][new_feature_names]
    xxx = pca.transform(xxx)
    X_train_2 = np.vstack((X_train, xxx))

    encoding_dim = 32  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

    # this is our input placeholder
    input_img = Input(shape=(X_train_2.shape[1],))
    # "encoded" is the encoded representation of the input
    encoded = Dense(encoding_dim, activation='relu')(input_img)
    # "decoded" is the lossy reconstruction of the input
    decoded = Dense(X_train_2.shape[1], activation='sigmoid')(encoded)

    # this model maps an input to its reconstruction
    autoencoder = Model(input_img, decoded)

    encoder = Model(input_img, encoded)
    # create a placeholder for an encoded (32-dimensional) input
    encoded_input = Input(shape=(encoding_dim,))
    # retrieve the last layer of the autoencoder model
    decoder_layer = autoencoder.layers[-1]
    # create the decoder model
    decoder = Model(encoded_input, decoder_layer(encoded_input))
    autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
    autoencoder.fit(X_train, X_train,
                    epochs=500,
                    batch_size=256,
                    shuffle=True, verbose=0)    

    X_train = np.hstack((X_train, encoder.predict(X_train)))
    X_test = np.hstack((X_test, encoder.predict(X_test)))
    return X_train, X_test

In [154]:
kfold = StratifiedKFold(n_splits=3, shuffle=True)
cvscores = []
y_train_2 = np.array(y_train_2)
X_train_2 = np.array(X_train_2)

for class_weight in [{1:0.51, 0:0.49}, {1:0.55, 0:0.45}]:
    for num_smote in [0.95]:
        for num_layers in [3]:
            for first_layer in [75]:
                for loops in [50]:
                    for each_layer in [12]:
                        num_layers = num_layers
                        first_layer = first_layer
                        each_layer = each_layer
                        num_smote = num_smote
                        n_pca = 0.99
                        loops = loops
                        scores_f = []
                        scores_p = []
                        scores_r = []
                        print("\n\nnum_layers", num_layers, "\nfirst_layer", first_layer, 
                                  "\neach_layer", each_layer, "\nnum_smote", num_smote,
                              "\nloops", loops, "\nclass_weight", class_weight)
                        for train, test in kfold.split(X_train_2, y_train_2):
                            
                            pca = PCA(n_components = n_pca)
                            X_train_3 = pca.fit_transform(X_train_2[train])
                            X_test_3 = pca.transform(X_train_2[test])
                            y_train_3 = y_train_2[train]
                            x, y = create_data(X_train_3, y_train_3)
                            created_X = x
                            created_y = y
                            for i in range(loops):
                                X_train_3, y_train_3 = shuffle(X_train_3, y_train_3)
                                x, y = create_data(X_train_3, y_train_3)
                                created_X = np.vstack((created_X, x))
                                created_y = np.hstack((created_y, y))
                            X_train_3 = np.vstack([created_X, X_train_3])
                            y_train_3 =  np.hstack([created_y, y_train_3])

                            sm = SMOTE(random_state=27)
                            X_train_res, y_train_res = sm.fit_sample(X_train_3, y_train_3.ravel())


                            n_cols = X_train_res.shape[1]

                            input_img = Input(shape=(n_cols,))

                            x = Dense(n_cols, activation='relu')(input_img)

                            encoded = Dense(n_cols//2, activation='relu')(x)
                            encoded = Dense(n_cols//4, activation='relu')(encoded)

                            y = Dense(n_cols//8, activation='relu')(x)

                            decoded = Dense(n_cols//4, activation='relu')(y)
                            decoded = Dense(n_cols//2, activation='relu')(decoded)

                            z = Dense(n_cols, activation='sigmoid')(decoded)
                            model = Model(input_img, z)

                            model.compile(optimizer='adadelta', loss='mse') # reporting the accuracy

                            xxx = df[df["PCL_Strict3"].isna()][new_feature_names]
                            xxx = pca.transform(xxx)
                            X_train_res_2 = np.vstack((X_train_res, xxx))

                            
                            model.fit(X_train_res_2, X_train_res_2,
                                  epochs=1000,
                                  batch_size=128,
                                  verbose=0)

                            mid = Model(input_img, y)
                            #reduced_representation =mid.predict(X_train_2[test])

                            out = Dense(1, activation='sigmoid')(y)
                            reduced = Model(input_img, out)
                            reduced.compile(loss='binary_crossentropy',
                                      optimizer='adam', 
                                      metrics=['accuracy']) 

                            reduced.fit(X_train_res, y_train_res,
                                  epochs=1000,
                                  batch_size=128,
                                  verbose=0, class_weight=class_weight)
                            
                            y_pred =  reduced.predict(X_test_3)
                        
                            
                            y_pred = (y_pred) > 0.5
                            
                            s_f = f1_score(y_pred,y_train_2[test])
                            s_p = precision_score(y_pred,y_train_2[test])
                            s_r = recall_score(y_pred,y_train_2[test])
                            print("\tscores f1", (s_f))
                            print("\tscores p", (s_p))
                            print("\tscores r", (s_r))
                            scores_f.append(s_f)
                            scores_p.append(s_p)
                            scores_r.append(s_r)

                        print("mean scores f1", np.mean(scores_f))
                        print("mean scores p", np.mean(scores_p))
                        print("mean scores r", np.mean(scores_r))
                       



num_layers 3 
first_layer 75 
each_layer 12 
num_smote 0.95 
loops 50 
class_weight {1: 0.51, 0: 0.49}
	scores f1 0.23076923076923078
	scores p 0.23076923076923078
	scores r 0.23076923076923078
	scores f1 0.32432432432432434
	scores p 0.46153846153846156
	scores r 0.25
	scores f1 0.1714285714285714
	scores p 0.25
	scores r 0.13043478260869565
mean scores f1 0.24217404217404218
mean scores p 0.3141025641025641
mean scores r 0.20373467112597546


num_layers 3 
first_layer 75 
each_layer 12 
num_smote 0.95 
loops 50 
class_weight {1: 0.55, 0: 0.45}
	scores f1 0.18181818181818185
	scores p 0.15384615384615385
	scores r 0.2222222222222222
	scores f1 0.21428571428571427
	scores p 0.23076923076923078
	scores r 0.2
	scores f1 0.0
	scores p 0.0
	scores r 0.0
mean scores f1 0.13203463203463203
mean scores p 0.12820512820512822
mean scores r 0.14074074074074075
